***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-051b  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import struct
import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
######CHANGE FOLDER NAME!!!!!!!!!!!!!!!!!!!!!!!!#########################################################
FOLDER_NAME =

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  5 10:41 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'idx316179928316366087idx' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

graphframes.sh
postings_small_anchor_text_gcp/0_000.bin
postings_small_anchor_text_gcp/0_posting_locs.pickle
postings_small_anchor_text_gcp/102_000.bin
postings_small_anchor_text_gcp/102_posting_locs.pickle
postings_small_anchor_text_gcp/103_000.bin
postings_small_anchor_text_gcp/103_posting_locs.pickle
postings_small_anchor_text_gcp/105_000.bin
postings_small_anchor_text_gcp/105_posting_locs.pickle
postings_small_anchor_text_gcp/106_000.bin
postings_small_anchor_text_gcp/106_posting_locs.pickle
postings_small_anchor_text_gcp/107_000.bin
postings_small_anchor_text_gcp/107_posting_locs.pickle
postings_small_anchor_text_gcp/108_000.bin
postings_small_anchor_text_gcp/108_posting_locs.pickle
postings_small_anchor_text_gcp/109_000.bin
postings_small_anchor_text_gcp/109_posting_locs.pickle
postings_small_anchor_text_gcp/113_000.bin
postings_small_anchor_text_gcp/113_posting_locs.pickle
postings_small_anchor_text_gcp/119_000.bin
postings_small_anchor_text_gcp/119_posting_locs.pickle
postings_

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [8]:
########################only for title###################################
# full_path = "gs://wikidata_preprocessed/*"
# parquetFile = spark.read.parquet(full_path)
# doc_text_pairs = parquetFile.select("title", "id").rdd

In [ ]:
###create id - title dictionary ---------- only run once
doc_text_pairs = parquetFile.select("id","title").rdd
id_2_title_dict = doc_text_pairs.collectAsMap()
with open(Path(".") / f'dict_of_id_title.pkl', 'wb') as f:
    pickle.dump(id_2_title_dict, f)
index_src = "dict_of_id_title.pkl"
index_dst = f'gs://{bucket_name}/{index_src}'
!gsutil cp $index_src $index_dst

In [9]:
#######################################only for anchor text###################################
def id_to_anchor_text(id_and_text):
  return [(id,text) for id, text in id_and_text]

def reduce_anchor_text(list_of_text):
  return " ".join(list_of_text)

####################COMMENT OUT WHEN CALCULATING ANCHOR TEXT INDEX ###########################################
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
x = pages_links.flatMap(lambda x : id_to_anchor_text(x[1]))
doc_text_pairs = x.groupByKey().mapValues(reduce_anchor_text).map(lambda x:(x[1],x[0]))


We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [10]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [11]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [13]:

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  #ADDED stopwords removal
  tokens = [token for token in tokens if token not in all_stopwords]
  # YOUR CODE HERE
  unique_tokens = set(tokens)
  if len(unique_tokens)==0:
      return []
  #### ADDED doc_len and max_tf to posting lists
  #return [(word,(id,tf,max_tf, doc_len)) for word,tf in count.items() if word not in all_stopwords]
  return [(word,id) for word in unique_tokens]



def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  return postings.mapValues(lambda x: len(x))


def word_posting2bucket_and_wpl(word, posting_list):
  return token2bucket_id(word),(word,posting_list)

def write_bucket_to_disk(bucket_id, list_of_words_postings_tuples):
  word2posting_locs = InvertedIndex().write_a_posting_list((bucket_id, list_of_words_postings_tuples),bucket_name, FOLDER_NAME)
  return word2posting_locs

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  bucket2posting_lists=postings.map(lambda x: word_posting2bucket_and_wpl(x[0],x[1])).groupByKey()#.mapValues().reduce_word_counts)
  x= bucket2posting_lists.map(lambda x: write_bucket_to_disk(x[0],x[1]))
  return x

In [15]:
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey()
# filtering postings and calculate df
####################################maybe don't need to reduce words##########################################
# postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings).collect()



In [16]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix=FOLDER_NAME):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [17]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
inverted.AVGDL = 79.8183118653278
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/{FOLDER_NAME}/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
/ [1 files][  2.2 KiB/  2.2 KiB]                                                
Operation completed over 1 objects/2.2 KiB.                                      


In [ ]:
!gsutil ls -lh $index_dst